# Accessing info with `qiskit-superstaq`
Below we'll go over how you can retrieve information on your account, your jobs, and backends that you can access with Superstaq.

## Imports and API Token

As usual, we'll being with importing requirements and setting up access to Superstaq.This tutorial uses `qiskit-superstaq`, our Superstaq client for Qiskit. You can install it and relevant dependencies by running `pip install qiskit-superstaq[examples]`.

In [ ]:
%pip install qiskit-superstaq[examples]

In [1]:
# requirements to use qiskit-superstaq
import qiskit
import qiskit_superstaq as qss

# optional
import json # not required to use SuperstaQ; used here to pretty print backend information
import os # used to set a token as an environment variable

Now, we instantiate a provider in `qiskit-superstaq` with `SuperstaQProvider()`. Supply the Superstaq API token by providing the token as an argument of `qss.SuperstaQProvider()` or setting it as an environment variable (see [this guide](/get_started/basics/basics_qss.html)).

In [2]:
# for qiskit-superstaq: provider for SuperstaQ backend
provider = qss.SuperstaQProvider()

## Account Information

The `provider` class gives you a means to retrieve and set SuperstaQ account information.
* `get_balance()`: 
* `ibmq_set_token()`: <mark><span style="color:red">doesn't work</span></mark>
* <mark><span style="color:red">Check user role - not part of Open Beta</span></mark>
    - If you'd like to upgrade your user account, please reach out to superstaq@infleqtion.com!

Below is a brief example of how to use one of these functions:

In [3]:
provider.get_balance()

'$76.25'

## Backend Information
In addition to account information, the`provider` object also gives you a list of all the devices to which you have access, <mark><span style="color:red">as well as additional information about those devices.</span></mark>
* <mark><span style="color:red">methods for device status, quantum volume, gate fidelities</span></mark>
* `backends()`: Retrieves a list of available backends
    - <mark><span style="color:red">filter backends?</span></mark>
* `get_backend("<backend_name>")`: Select your target backend, where `<backend_name>` is the name of the desired device
* 

In [27]:
backend = provider.get_backend("ibmq_lagos_qpu")
backend.target_info()

{'num_qubits': 7,
 'target': 'ibmq_lagos_qpu',
 'native_gate_set': ['id', 'rz', 'sx', 'x', 'cx', 'reset'],
 'max_experiments': 300,
 'coupling_map': [[0, 1],
  [1, 0],
  [1, 2],
  [1, 3],
  [2, 1],
  [3, 1],
  [3, 5],
  [4, 5],
  [5, 3],
  [5, 4],
  [5, 6],
  [6, 5]],
 'max_shots': 32000,
 'processor_type': {'family': 'Falcon', 'revision': '5.11', 'segment': 'H'},
 'open_pulse': True}

In [28]:
backend.status().to_dict()

{'backend_name': 'ibmq_lagos_qpu',
 'backend_version': '1',
 'operational': True,
 'pending_jobs': 0,
 'status_msg': ''}

In [30]:
provider.backends() # check in qss to see which method this should be instead to get list of compile only, etc.

AttributeError: 'SuperstaQProvider' object has no attribute 'get_backends'

## Job Information
Jobs submitted through SuperstaQ contain the following information:
* `job_id()`: Unique identifier for the job
* `status()`: Status of the job (either <mark><span style="color:red">Queued, Running, Done</span></mark>)
* `backend()`: Device the job was run on
* `result().get_counts()`: Counts from the result of the job run
* circuit? tag for a job, retrieve jobs by user/tag

In [3]:
# using Qiskit
qc = qiskit.QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

backend = provider.get_backend("ibmq_qasm_simulator")
job_qss = backend.run(qc, shots=100)
results_qss = job_qss.result().get_counts()


In [7]:
print(job_qss.result())

Result(backend_name='ibmq_qasm_simulator', backend_version='n/a', qobj_id='-1', job_id='daec9b20-f097-443f-9b0e-103a0aa8fd9e', success=True, results=[ExperimentResult(shots=100, success=True, meas_level=2, data=ExperimentResultData(counts={'00': 53, '11': 47}))], date=None, status=None, header=None)
